<a href="https://colab.research.google.com/github/zukol/dw_3miasto_project/blob/master/scraper_lento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import time
import requests
from slugify import slugify
from bs4 import BeautifulSoup
import json
import pandas as pd
import random


In [48]:
class PageScrapper:

    def __init__(self, url):

        self.url = url
        self.last_page = self.get_last_page_number() #pobiera ile jest stron z ogloszeniami

        print(f"Total number of pages: {self.last_page}")

    def get_last_page_number(self):
        page = self.read_page_content(self.url)
        last_page_element = page.find('span', {"class": "hash number alike button"}).get_text() #znajdujemy element span z klasa nazywajaca sie hash number alike button i pobieramy z tego tekst
        return int(last_page_element)
    

    def find_advertisements(self):

        for page_number in range(self.last_page + 1): #iterujemy od 1 od ostatniej strony

            print(f"Page {page_number+1}/{self.last_page} processing...") # pokazuje na ktorej aktualnie stronie jetesmy

            page = self.read_page_content(self.url + f'&page={page_number}') #dopisuje do adresu url kawalek adresu z nr strony
            advertisements = page.findAll("a", {"class": "title-list-item"}) #wyszukujemy wszystkie nazwy ogloszen

            for advertisement_index in range(len(advertisements)):
                data = self.parse_advertisement(advertisements[advertisement_index]['href']) #wchodzimy w petli pokolei w ogloszenia i czytamy tresc za pomaca parse_advertisement
                print(data)
                sleep_time = random.uniform(0.2, 5) #generowanie float z przedzialu od 0.2 do 5
                time.sleep(sleep_time) #przerwa zeby nie zostac zablokowanym

    def parse_advertisement(self, url):

        data = {}

        page = self.read_page_content(url)
        container = page.find('div', {'class': 'container-fluid otherpage-container'}) #pobieramy cala strone ogloszenia, ale bez reklam, menu, itp
        parameters = container.findAll('div', {'class': 'details text-15'}) #wskazujemy gdzie znajduja sie parametry

        data['data'] = container.find('div', {'class': 'pull-left text-13'}).get_text()
        #data['tresc'] = container.find('div', {'class': 'desc text-15'}).get_text().strip() #pobieramy tresc ogloszenia
        data['cena'] = float(container.find('span', {'class': 'price'})
                             .get_text().replace('zł', '').replace(' ', '').replace(',', '.')) #usuwamy zl spacje zastepujemy , na .
        data['cena_za_metr'] = float(container.find('span', {'class': 'text-g'})
                             .get_text().replace('zł/m2', '').replace(' ', '').replace(',', '.'))                     
        # try:
        #   data['liczba_zdjec'] = container.find('div', {'class': 'gallery-counter licon-gallery-f'}).get_text
        # except AttributeError:
        #   pass  
        data['tytul'] = container.find('h2').get_text().strip() #pobieramy nazwe ogloszenia za pomoca naglowka h2, metoda strip usuwa biale znaki
        data['adres'] = container.find('div', {'class': 'userbox-location-city licon-pin-f'}).get_text()
        data['wysweitlenia'] = container.find('span', {'class': 'text-b'}).get_text()
        
        
        for parameter_index in range(len(parameters)):
            label = parameters[parameter_index].find('span', {'class': 'label'}) #dla kazdego elementu li odczytujemy label i value
            value = parameters[parameter_index].find('span', {'class': 'row-old'})

            if value is not None:
                data[slugify(label.get_text())] = value.get_text() # biblioteka ktora tworzy adresy url z tytulow, usuwa polskie znaki itp

        return data

    def read_page_content(self, url):
        page = requests.get(url)
        return BeautifulSoup(page.content, "html.parser")

    def create_json(self):
        with open('data_lento.json', 'a') as fs:
            json.dump(data, fs)
   

URL = 'https://sopot.lento.pl/nieruchomosci/mieszkania/sprzedaz.html?radius=15&price_from=100000'
scraper = PageScrapper(URL)
scraper.find_advertisements()
scraper.create_json()


Total number of pages: 19
Page 1/19 processing...
{'data': 'Dodane: Poniedziałek, 11 Maj, 2020 \xa015:19', 'cena': 439000.0, 'cena_za_metr': 7317.0, 'tytul': 'Sprzedam mieszkanie 60,32m2+miejsce postojowe w cenie', 'adres': 'Gdańsk / Gdańsk Południe', 'wysweitlenia': '245', 'kategoria': 'Mieszkania / Sprzedaż'}
{'data': 'Dodane: Czwartek, 07 Maj, 2020 \xa012:06', 'cena': 640000.0, 'cena_za_metr': 8649.0, 'tytul': 'Komfortowe, 3 pokojowe, w pełni wyposażone, Gdynia', 'adres': 'Gdynia', 'wysweitlenia': '613', 'kategoria': 'Mieszkania / Sprzedaż'}


KeyboardInterrupt: ignored

In [0]:
!pwd

/content/drive/My Drive/Colab Notebooks/dw_3miasto/data


In [0]:
mkdir -p '/content/drive/My Drive/Colab Notebooks/dw_3miasto/data'

In [0]:
cd '/content/drive/My Drive/Colab Notebooks/dw_3miasto/data'

/content/drive/My Drive/Colab Notebooks/dw_3miasto/data


In [0]:
pwd

'/content/drive/My Drive/Colab Notebooks/dw_3miasto/data'

In [0]:
ls

In [0]:
df = pd.read_json('data_lento.json', lines=True)
df

ValueError: ignored